# Exploratory Data Analysis

In [ ]:
%%capture
!pip install pandas
!pip install geopandas
!pip install plotly-express
!pip install nbformat
!pip install -U kaleido
!pip install pycountry-convert

In [ ]:
import plotly.express as px
import pandas as pd
import pycountry_convert as pc

We first add continent information to our data frame

In [ ]:
def f(x): return(pc.country_alpha2_to_continent_code(pc.country_alpha3_to_country_alpha2(x)))

df = pd.read_csv('../Data/global_average_yearly_temp_with_features_clean.csv')
df['datetime_year'] = pd.to_datetime(df['year'], format = "%Y")
df['iso_continents_code'] = df['iso_code'].apply(f) 


## Global Trends
We now see the general global trends of temperature overtime has been increasing. When we plot each continents temperature, we notice that the data has been most consistent for Europe, Asia, North America and South America, but Africa has sparse data before 1850.

### Global Average Temperature

In [ ]:
year_group_by = df.groupby("year").mean()
year_group_by = year_group_by.reset_index()
fig = px.line(year_group_by, x = "year", y="AvgYearlyTemp", title = "Average Temperature Over Time")
fig.show()


### Continent Average Temperature 

In [ ]:
continent_group_by = df.groupby(["year","iso_continents_code"]).mean()
continent_group_by = continent_group_by.reset_index()
fig = px.line(continent_group_by, x = "year", y="AvgYearlyTemp", color = "iso_continents_code")
fig.show()

### Map of the World With Average Temperature

In [ ]:
fig = px.choropleth(df, locations="iso_code", color= "AvgYearlyTemp", 
                    hover_name= "iso_code", animation_frame= "year", title = "Average Yearly Temperature" )
# interactive
fig.show()

### Percent Change In Temperature Year Over Year 

In [ ]:
year_group_by.sort_values(['year'], inplace = True, ascending=[False])
temp_label, co2_label = 'Avg. Temperature Yearly Percent Change', 'Avg. Co2 Yearly Percent Change'
year_group_by[temp_label] = year_group_by['AvgYearlyTemp'].pct_change() * 100
year_group_by[co2_label] = year_group_by['co2'].pct_change() * 100
fig = px.bar(year_group_by, x='year', y=temp_label, color = temp_label, color_continuous_scale ='bluered')
fig.show()

## Trends Between Data

In [ ]:
corr = df.corr()
fig = px.imshow(corr, True, title = "Correlation Heatmap")
fig.show()

In [ ]:
fig = px.scatter_matrix(df, dimensions=["cumulative_co2", "population", "co2", "oil_co2", "AvgYearlyTemp"])
# interactive plot
fig.show()

# Creating Sarimax Model With Excog Variables At A Country Level

We will be creating a Sarimax model at country level because each time forecasting relies on the trends in data, therefore you cannot have multiple instances for a single time point. Moreover, as a simple model, it only requires not many datapoints to make predictions. Below, we detail the process that we have for our model.

1. Data Preperation
2. Hypertuning Model Parameters With Training and Validation Set
3. Testing model on holdout set


In [ ]:
%%capture
!pip install pandas
!pip install plotly-express
!pip install statsmodels
!pip install tqdm


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import statsmodels as sm 
from sklearn.metrics import r2_score
from tqdm import tqdm
import plotly.express as px
from sklearn.preprocessing import StandardScaler
# model imports
from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm1
from sklearn.metrics import mean_squared_error, r2_score
import itertools
import math
from sklearn.decomposition import PCA
import pickle


## Preparing Data for Arima
We have multple task to prepare data for ARIMA
### Data Wrangling
1. We have to cast our Year into a datetime object
2. We then have to set it as an index of the array
3. Finally, we have to set the period of the data to Yearly frequency

### Spliting Data into Training, Validation, and Holdout
We selection the following time periods.
```
Training (1750 - 1950)
Validation (1950 - 1980)
Testing (1980 - 2013)
```

### Scaling Data By Standard Scalar
The numerical method used for ARIMA involves BFGS, which is not
scale invariant, therfore we scale the x-features by that of
the training data

In [2]:
def get_country_df(country):
    df = pd.read_csv('../Data/global_average_yearly_temp_with_features_clean.csv')
    df = df[df["iso_code"] == country]
    df["Year_idx"] = pd.to_datetime(df.year, format="%Y")
    df = df.set_index("Year_idx")
    df.index = df.index.to_period("Y")
    return df

def get_test_train_valid(country, train_split = '1950', validation_split = '1980'):
    country_df = get_country_df(country)
    
    train = country_df.loc[:train_split]
    valid = country_df.loc[train_split: validation_split]
    test  = country_df.loc[validation_split:]
    
    return train, test, valid

def scale(train_x, valid_x, test_x):
    scaler = StandardScaler()
    model_x = pd.concat([train_x, valid_x])

    scaler.fit(model_x)
    model_x = scaler.transform(model_x)
    train_x = scaler.transform(train_x)
    valid_x = scaler.transform(valid_x)
    test_x  = scaler.transform(test_x)
    return train_x, valid_x, test_x, model_x

def pca_fitter(train_x, threshold = .99):
    n_features = train_x.shape[1]
    
    for n in range(1, n_features + 1):
        pca_model = PCA(n)
        pca_model.fit(train_x)
        
        if sum(pca_model.explained_variance_ratio_) >= threshold:
            break
    return pca_model
        
    

## Hypertuning Model Parameters With Training and Validation Set
We opt for an exhaustive grid search to find the parameters that give the best result. Here, we only use the following assumptions for the sarimax model. The external regressors are time varying, and have measurement error. 

### Parameter Scanning
**p** is the number of autoregressive terms

**d** is the number of nonseasonal differences needed for stationarity 

**q** is the number of lagged forecast errors in the prediction equation, also known as Moving Average Term

We define Y as the random variable that is trying to be predicted. So, d determines dth difference of Y. Moreover, we define $y_t, w_{t}, x_k$ as the previous value, white noise process, and kth regressor respectively. $\phi_i, \theta_i, a_k$ are coefficients assoicated with the parameters above.
$$
a_t = \sum_{i=1}^p \alpha_i y_{t-i} + w_t\\
m_t = w_t + \sum_{i = 1}^{q} \beta_i w_{t - i}\\
d_t = (y_t - y_{t - 1}) - (y_{t - 1} - y_{t - 2}) - ... - (y_{t-(d-1)} - y_{t-d})\\
l_t = \sum_{i = 0}^n a_i x_i 
$$
With these terms defined, the model is as follows.
$$
a_t*d_t - m_t + l_t = 0
$$



In [3]:
def eval_sarimax_excog(train_x, train_y, test_x, test_y, arima_order,\
                       enforce_invertibility, enforce_stationarity):
    # USE THIS
    model = sm1.tsa.statespace.SARIMAX(train_y, exog = train_x, order = arima_order, \
                                       time_varying_regression = True, mle_regression = False,\
                                       measurement_error = True, enforce_invertibility = enforce_invertibility,\
                                       enforce_stationarity = enforce_stationarity
                                      )
    model_fit = model.fit(disp = 0)
    predictions = model_fit.forecast(len(test_x), exog = test_x)
    rmse = math.sqrt(mean_squared_error(test_y, predictions))
    predictions.index = test_y.index
    return model_fit, rmse, predictions

def eval_excog_models_by_rmse(train_x, train_y, test_x, test_y, p_values, d_values, q_values, f):
    arima_orders = itertools.product(*[p_values, d_values, q_values])
    arima_orders = list(arima_orders)
    best_order, best_score = None, float("inf") 
    best_enforce_invertibility = None
    best_stationary = None
    count = 0
    for arima_order in tqdm(arima_orders):
        for enforce_invertibility in [True, False]:
            for enforce_stationarity in [True, False]:
                try:
                    _, rmse, _ = f(train_x, train_y, test_x, test_y, arima_order, \
                                   enforce_invertibility = enforce_invertibility,\
                                   enforce_stationarity = enforce_stationarity)

                    if rmse < best_score:
                        best_score, best_order = rmse, arima_order
                        best_enforce_invertibility = enforce_invertibility
                        best_stationary= enforce_stationarity
                        print(f"ARIMA RMSE = {best_score}")
                except Exception as e:
                    print(e)
                    pass
                count += 1
    print('DONE')
    return best_order, best_enforce_invertibility, best_stationary

def eval_excog_models_by_aic(train_x, train_y, test_x, test_y, p_values, d_values, q_values, f):
    arima_orders = itertools.product(*[p_values, d_values, q_values])
    arima_orders = list(arima_orders)
    best_order, best_score = None, float("inf") 
    best_enforce_invertibility = None
    best_stationary = None
    count = 0
    for arima_order in tqdm(arima_orders):
        for enforce_invertibility in [True, False]:
            for enforce_stationarity in [True, False]:
                try:
                    model_fit, _, _ = f(train_x, train_y, test_x, test_y, arima_order, \
                                   enforce_invertibility = enforce_invertibility,\
                                   enforce_stationarity = enforce_stationarity)

                    if model_fit.aic < best_score:
                        best_score, best_order = model_fit.aic, arima_order
                        best_enforce_invertibility = enforce_invertibility
                        best_stationary= enforce_stationarity
                        print(f"ARIMA AIC = {best_score}")
                except Exception as e:
                    print(e)
                    pass
                count += 1
    print('DONE')
    return best_order, best_enforce_invertibility, best_stationary

In [61]:
def ml(country, f = eval_excog_models_by_rmse, p_values = [0, 1, 2], d_values = [0, 1, 2], q_values = [0, 1, 2]):
    train, test, valid = get_test_train_valid(country)

    train_x, train_y = train[train.columns[2:-2]], train["AvgYearlyTemp"]
    test_x, test_y = test[test.columns[2:-2]], test["AvgYearlyTemp"]
    valid_x, valid_y = valid[valid.columns[2:-2]], valid["AvgYearlyTemp"]

    train_x, valid_x, test_x, model_x = scale(train_x, valid_x, test_x)

    pca_model = pca_fitter(train_x)
    train_x = pca_model.transform(train_x)
    valid_x = pca_model.transform(valid_x) 
    test_x  = pca_model.transform(test_x)
    model_x = pca_model.transform(model_x)
    
    model_y = pd.concat([train_y, valid_y])

    best_order, best_enforce_invertibility, best_stationary = f(train_x, train_y, valid_x, valid_y, p_values, d_values, q_values, eval_sarimax_excog)

    model_fit, rmse, predictions = eval_sarimax_excog(model_x, model_y, test_x, test_y, best_order, best_enforce_invertibility, best_stationary)
    
    return best_order, best_enforce_invertibility, best_stationary, model_fit, test_y, predictions

In [100]:
def plot(df, country, layout, line1 = 'AvgYearlyTemp', line2 = 'Predictions', x = 'year', error = 'AvgTempUncertainty'):
    from plotly.subplots import make_subplots
    import plotly.graph_objects as go


    fig = go.Figure(layout = layout)
    trace_1 = go.Line(name = f"Actual {country} Avg Yearly Temp", 
                     x = df[x],
                     y = df[line1],
                     error_y = dict(type='data', array = test[error]))


    trace_2 = go.Line(name = f"Predictions of {country} Avg Yearly Temp", 
                     x = df[x],
                     y = df[line2])

    fig.add_trace(trace_1)
    fig.add_trace(trace_2)
    
    return fig

## Testing on the holdout
We now use the best parameters from an exhaustive grid search to predict on our holdout set

In [68]:
%%capture
results = {"FRA": None, "DEU": None, 
           "CAN": None, "ESP": None, 
           "IND": None}

for k in results.keys():
    results[k] = ml(k, f= eval_excog_models_by_aic)


In [107]:
import pickle
from sklearn.metrics import mean_absolute_error

In [70]:

with open('Temperatue_Time_Series.pickle', 'wb') as fh:
   pickle.dump(results, fh)

In [71]:
pickle_off = open ("Temperatue_Time_Series.pickle", "rb")
results = pickle.load(pickle_off)

In [111]:
import os

if not os.path.exists("images"):
    os.mkdir("images")
    
for k, v in results.items():
    train, test, valid = get_test_train_valid(k)
    
    p, q, d = v[0]

    model = pd.concat([train, valid])
    model['Predictions'] = v[3].forecasts[0]
    x, y = model['Predictions'], model['AvgYearlyTemp']
    r2  = r2_score(x, y)
    mse = mean_squared_error(x, y)
    mae = mean_absolute_error(x, y)
    layout = dict(title = f'Training Set-- {k} P: {p} Q: {q} D: {d}  MAE: {round(mae, 2)} MSE: {round(mse, 2)}',
          xaxis=dict(title='Year'),
          yaxis=dict(title='Temperature'))
    
    fig = plot(model, k, layout)
    fig.show()
    p, q, d = v[0]
    test['Predictions'] = v[-1]
    
    x, y = test['Predictions'], test['AvgYearlyTemp']
    r2  = r2_score(x, y)
    mse = mean_squared_error(x, y)
    mae = mean_absolute_error(x, y)
    
    layout = dict(title = f'Holdout Set-- {k} P: {p} Q: {q} D: {d} MAE: {round(mae, 2)} MSE: {round(mse, 2)}',
              xaxis=dict(title='Year'),
              yaxis=dict(title='Temperature'))
    
    fig = plot(test, k, layout)
    fig.write_image(f"images/holdout_{k}_p{p}_q{q}_d{d}.png")
    fig.show()

/home/sumants/miniconda3/envs/watts/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/home/sumants/miniconda3/envs/watts/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/home/sumants/miniconda3/envs/watts/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/home/sumants/miniconda3/envs/watts/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/home/sumants/miniconda3/envs/watts/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/home/sumants/miniconda3/envs/watts/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/home/sumants/miniconda3/envs/watts/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/home/sumants/miniconda3/envs/watts/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/home/sumants/miniconda3/envs/watts/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




/home/sumants/miniconda3/envs/watts/lib/python3.7/site-packages/plotly/graph_objs/_deprecations.py:385: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.


